In [1]:
import numpy as np
import torch as t
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib widget
from ipywidgets import FloatSlider, interact, interact_manual

$$
\newcommand{\bracket}[3]{\left#1 #3 \right#2}
\newcommand{\b}{\bracket{(}{)}}
\newcommand{\Bernoulli}{{\rm Bernoulli}\b}
\newcommand{\x}{\mathbf{x}}
\newcommand{\X}{\mathbf{X}}
\newcommand{\m}{\boldsymbol{\mu}}
\newcommand{\P}{{\rm P}\b}
\newcommand{\dd}[2][]{\frac{\partial #1}{\partial #2}}
\newcommand{\S}{\mathbf{\Sigma}}
\newcommand{\Sh}{\mathbf{\hat{\Sigma}}}
\newcommand{\mh}{\boldsymbol{\hat{\mu}}}
\newcommand{\N}{\mathcal{N}\b}
\newcommand{\det}{\bracket{\lvert}{\rvert}}
\newcommand{\sb}{\bracket{[}{]}}
\newcommand{\E}{\mathbb{E}\sb}
\newcommand{\Var}{{\rm Var}\sb}
\newcommand{\Cov}{{\rm Cov}\sb}
\DeclareMathOperator*{\argmax}{arg\,max}
\DeclareMathOperator*{\argmin}{arg\,min}
\newcommand{\ph}{\hat{p}}
\newcommand{\at}{\bracket{.}{\rvert}}
\newcommand{\w}{\mathbf{w}}
\newcommand{\W}{\mathbf{W}}
\newcommand{\W}{\mathbf{W}}
\newcommand{\Wh}{\mathbf{\hat{W}}}
\newcommand{\Y}{\mathbf{Y}}
\newcommand{\L}{\mathcal{L}}
\newcommand{\wh}{\mathbf{\hat{w}}}
\newcommand{\y}{\mathbf{y}}
\newcommand{\0}{\mathbf{0}}
\newcommand{\I}{\mathbf{I}}
$$

<h1> Lecture 1: Supervised learning: classification and regression </h1>

First things first:
<ul>
  <li> I'm experimenting with doing the lectures in Jupyter Notebooks.</li>
  <li> Hopefully, it means we can connect theory and code more closely.</li>
  <li> I'm going to use the PyTorch distributions library, which is much better than that from Numpy/Scipy.</li>
  <li> Don't be scared of PyTorch: the syntax is almost exactly the same as Numpy.</li>
  <li> There may be some repetition from previous lectures, but I'll be doing things much more deeply, with more simulations</li>
</ul>

In supervised learning, we have a bunch of input, $x$, output, $y$, pairs that are data.

The great thing about supervised learning is that the inputs and outputs could be almost anything,
```
x :: Image,  y :: Int               # Object recognition
x :: Image,  y :: Matrix{Int}       # Image segmentation
x :: Audio,  y :: Str               # Speech recognition
x :; Str,    y :: Audio             # Text-to-speech
```
The goal is to learn something about the mapping from $x$ to $y$.

One approach is to learn a function that maps from $x$ to a guess about the corresponding $y$.  This guess is called $\hat{y}$,

\begin{align}
  f(x) \rightarrow \hat{y}
\end{align}

To learn $f$, we try to make $y$ from the data as similar as possible to the estimates, $\hat{y}$.

However, to choose this function, we require a measure of "similiarity", and it isn't always possible to give one.

Instead, we usually write a function that takes an $x$ and returns a distribution over $y$,

\begin{align}
  f(x) \rightarrow \P{y| x}
\end{align}

It is easier to fit such a distribution, because we maximise the probability of the $y$ that we saw in the data.

Note: classification and regression are special types of supervised learning.  In classification, the output is, a class-label,
```
Y = Int     #classification
```
in regression, the output is one (or many) real values,
```
Y = Float   #regression
```
If the output is more complicated (e.g. a string, image or audio), then its neither regression or classification, its just supervised learning.

<h2> Multivariate linear regression </h2>

The most useful and instructive supervised learning method is multivariate linear regression.

The input for the $\lambda$th data point is, $\x$, is a vector and we consider the multi-output case, where the corresponding output, $\y_\lambda$ is also a vector.
We assume that $\y_\lambda$ is Gaussian, conditioned on $\x$,

\begin{align}
  \P{\y_{\lambda}| \x_\lambda, \W} &= \N{\y_{\lambda}; \x_{\lambda} \W^T, \sigma^2 \I}
\end{align}

Note, here, I'm treating. $\y_\lambda$ and $\x_\lambda$ as row-vectors, not column vectors, as is standard in maths.  Turns out this allows us to almost directly translate maths to Numpy/PyTorch, so this it really helps for us!

If the covariance is diagonal, then we can write the distribution as a Gaussian over each element, $Y_{\lambda i}$ separately,

\begin{align}
  \P{Y_{\lambda i}| \X, \W} &= \N{Y_{\lambda i}; \sum_j X_{\lambda j} W_{i j}, \sigma^2 \I}
\end{align}

We're going to work with the multi-output case, as it makes the choice of different dimensions/transposes a bit more obvious,

\begin{align}
  \L\b{\W} &= \sum_{\lambda i} \log \P{Y_{\lambda i}| \X, \W}\\
  \L\b{\W} &= \sum_{\lambda i} \sb{ -\tfrac{1}{2} \log 2 \pi \sigma^2 - \tfrac{1}{2 \sigma^2} \b{Y_{\lambda i} - \sum_j X_{\lambda j} W_{ij}}^2}.
\end{align}

The maximum likelihood estimate, $\Wh$, is the maximum of the log-likelihood,

\begin{align}
  \Wh &= \argmax_\W \L\b{\W}
\end{align}

To compute the maximum, we take the gradient,

\begin{align}
  \dd[\L\b{\W}]{W_{\alpha \beta}} &= - \frac{1}{2 \sigma^2} \dd{W_{\alpha \beta}} \sum_{\lambda i} \b{Y_{\lambda i} - \sum_j X_{\lambda j} W_{ij}}^2.
\end{align}

This looks hard.  But because we've written it in index notation, we can do everything in terms of standard calculus.  We start by applying the chain rule,

\begin{align}
  \dd[\L\b{\W}]{W_{\alpha \beta}} &= - \frac{1}{\sigma^2} \sum_{\lambda i} \b{Y_{\lambda i} - \sum_j X_{\lambda j} W_{ij}} \dd{W_{\alpha \beta}} \b{Y_{\lambda i} - \sum_j X_{\lambda j} W_{ij}}.
\end{align}

interlude: the Kronecker delta (which looks alot like an identity matrix),

\begin{align}
  \dd[W_{ji}]{W_{\alpha \beta}} = \delta_{i \alpha} \delta_{j \beta}\\
  \delta_{ij} = \begin{cases}
    1 & \text{ if } i = j\\
    0 & \text{ if } i \neq j
  \end{cases}
\end{align}

thus,

\begin{align}
  \dd[\L\b{\W}]{W_{\alpha \beta}} &= \frac{1}{\sigma^2} \sum_{\lambda i} \b{Y_{\lambda i} - \sum_j X_{\lambda j} W_{ij}} \sum_j X_{\lambda j} \delta_{i \alpha} \delta_{j \beta}.
\end{align}

rearranging, so the $\delta$'s are next to the corresponding sum,

\begin{align}
  \dd[\L\b{\W}]{W_{\alpha \beta}} &= \frac{1}{\sigma^2} \sum_\lambda \underbrace{\sum_i \delta_{i \alpha}}_{\text{set } i=\alpha} \b{Y_{\lambda i} - \sum_j X_{\lambda j} W_{ij}} \underbrace{\sum_j \delta_{j \beta}}_{\text{set } j=\beta} X_{\lambda j}.
\end{align}

the $\delta_{i \alpha}$ picks out the $i=\alpha$ term in the sum, and the $\delta_{j \beta}$ picks out the $j=\beta$ term in the sum,

\begin{align}
  \dd[\L\b{\W}]{W_{\alpha \beta}} &= \frac{1}{\sigma^2} \sum_{\lambda} \b{Y_{\lambda \alpha} - \sum_j X_{\lambda j} W_{\alpha j}} X_{\lambda \beta}.
\end{align}

Finally, put everything back in vector/matrix notation,

\begin{align}
  \dd[\L\b{\W}]{\W} &= \tfrac{1}{\sigma^2} \b{\Y - \X \W^T}^T \X
\end{align}

We could just follow the gradient uphill (and that's what we do in deep learning), but this is super-slow.  Instead, there's an answer that is much faster to compute: at the top of the hill, the gradient is zero,

\begin{align}
  \0 &= \at{\dd[\L\b{\W}]{\W}}_{\W=\Wh}\\ 
  \0 &= \tfrac{1}{\sigma^2} \b{\Y - \X \Wh^T}^T \X \\
  \0 &= \b{\Y^T - \Wh \X^T} \X\\
  \Wh \X^T \X &= \Y^T \X
\end{align}

Note that we can't just solve for $\Wh$ using the inverse of $\X^T$ because it is almost never square, so the inverse does not exist!  As, $\X^T \X$ is square, we can take its inverse (note that $\X^T$ isn't square, so we can't take its inverse,)

\begin{align}
  \Wh  &= \Y^T \X \b{\X^T \X}^{-1}
\end{align}


I've done it this way, because the translation into Numpy/PyTorch is most direct, but if you look things up, you will often find the transposes in different places.

Lets write some code!  First, we can directly translate the above expression to give a method for fitting $\Wh$,

In [2]:
def fit_Wh(X, Y):
    return  (Y.T @ X) @ t.inverse(X.T @ X)

Now we generate some 1D "fake data"

In [3]:
N     = 100 # number of datapoints
D     = 1   # dimension of datapoints
sigma = 0.1 # output noise
X     = t.randn(N, D)
Wtrue = t.ones(1, D)
Y     = X @ Wtrue.T + sigma*t.randn(N, 1)

fig, ax = plt.subplots()
ax.set_xlabel("$x_\lambda$")
ax.set_ylabel("$y_\lambda$")
ax.scatter(X, Y);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [45]:
Wh = fit_Wh(X, Y)
print(f"Wtrue = {Wtrue}")
print(f"Wh    = {Wh}")

Wtrue = tensor([[1.]])
Wh    = tensor([[0.9909]])


In [46]:
fig, ax = plt.subplots()
ax.set_xlabel("$x_\lambda$")
ax.set_ylabel("$y_\lambda$")
ax.scatter(X, Y, label="data")
ax.plot(X, X@Wh.T, 'r', label="fitted line")
ax.legend();

/Users/laurence/programs/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  """Entry point for launching an IPython kernel.


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Now we can also generate some 2D "fake data",

In [47]:
N     = 100 # number of datapoints
D     = 2   # dimension of datapoints
sigma = 0.3 # output noise
X     = t.randn(N, D)
Wtrue = t.ones(1, D)
Y     = X @ Wtrue.T + sigma*t.randn(N, 1)

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.set_xlabel("$x_{\lambda, 0}$")
ax.set_ylabel("$x_{\lambda, 1}$")
ax.set_zlabel("$y_{\lambda, 0}$")
ax.scatter(xs=X[:, 0], ys=X[:, 1], zs=Y[:, 0]);

/Users/laurence/programs/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [48]:
Wh = fit_Wh(X, Y)
print(f"Wtrue = {Wtrue}")
print(f"Wh    = {Wh}")

Wtrue = tensor([[1., 1.]])
Wh    = tensor([[0.9781, 0.9921]])


In [49]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.set_xlabel("$x_{\lambda, 0}$")
ax.set_ylabel("$x_{\lambda, 1}$")
ax.set_zlabel("$y_{\lambda, 0}$")
ax.scatter(X[:, 0], X[:, 1], Y[:, 0])

Xp = t.tensor([
    [-4., -4.],
    [-4.,  4.],
    [ 4., -4.],
    [ 4.,  4.]
])

ax.plot_trisurf(
    np.array(Xp[:, 0]), 
    np.array(Xp[:, 1]), 
    np.array((Xp @ Wh.T)[:, 0]), 
    color='r', 
    alpha=0.3
);

/Users/laurence/programs/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  """Entry point for launching an IPython kernel.


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

But there are issues with linear regression, as formulated thus far.

In particular, consider 1D data, generated with a bias,

\begin{align}
  \P{y_\lambda| \x_\lambda, w, b} &= \N{\y_\lambda; x_\lambda w + b, \sigma^2}
\end{align}

If we fit our old model, without a bias, it doesn't work,

In [50]:
N     = 100 # number of datapoints
D     = 1   # dimension of datapoints
sigma = 0.1 # output noise
X     = t.rand(N, D)
Wtrue = 2*t.ones(1, D)
btrue = 3
Y     = X @ Wtrue + btrue + sigma*t.randn(N, 1)

fig, ax = plt.subplots()
ax.set_xlabel("$x_\lambda$")
ax.set_ylabel("$y_\lambda$")
ax.set_xlim(0, 1)
ax.set_ylim(0, 7)
ax.scatter(X, Y);

/Users/laurence/programs/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  if __name__ == '__main__':


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [51]:
Wh = fit_Wh(X, Y)
print(f"Wtrue = {Wtrue}")
print(f"Wh    = {Wh}")

Wtrue = tensor([[2.]])
Wh    = tensor([[6.6734]])


In [52]:
fig, ax = plt.subplots()
ax.set_xlabel("$x_\lambda$")
ax.set_ylabel("$y_\lambda$")
ax.set_xlim(0, 1)
ax.set_ylim(0, 7)
ax.scatter(X, Y, label="data")
ax.plot(X, X@Wh.T, 'r', label="fitted line")
ax.legend();

/Users/laurence/programs/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  """Entry point for launching an IPython kernel.


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

What can we do?  Well, we could go through the big derivation above again, incorporating the bias.

But this is super-tedious.

Instead, note that if we expand the 1D feature vector into a 2D feature vector, with the second feature being just biases,

\begin{align}
  \P{y_\lambda| \x_\lambda, w, b} &= \N{\y_\lambda; \underbrace{\begin{pmatrix} x_\lambda & 1 \end{pmatrix}}_\text{expanded feature vector} \overbrace{\begin{pmatrix} w \\ b \end{pmatrix}}^\text{expanded weight vector}, \sigma^2}.
\end{align}

Now, we can just use our original derivation, and implementation!

In [53]:
def add_bias(X):
    return t.cat([X, t.ones(X.shape[0], 1)], 1)

Xe = add_bias(X)
Xe[:10, :]

tensor([[0.7622, 1.0000],
        [0.7748, 1.0000],
        [0.9505, 1.0000],
        [0.0803, 1.0000],
        [0.7271, 1.0000],
        [0.2640, 1.0000],
        [0.8015, 1.0000],
        [0.1399, 1.0000],
        [0.1650, 1.0000],
        [0.4301, 1.0000]])

In [54]:
Wh = fit_Wh(Xe, Y)
print(f"Wh    = {Wh}")
print(f"Wtrue = {Wtrue}")
print(f"btrue = {btrue}")

Wh    = tensor([[1.9719, 3.0323]])
Wtrue = tensor([[2.]])
btrue = 3


In [55]:
fig, ax = plt.subplots()
ax.set_xlabel("$x_\lambda$")
ax.set_ylabel("$y_\lambda$")
ax.set_xlim(0, 1)
ax.set_ylim(0, 7)
ax.scatter(X, Y, label="data")

xs = t.tensor([[0.], [1.]])
ax.plot(xs, add_bias(xs)@Wh.T, 'r', label="fitted line")
ax.legend();

/Users/laurence/programs/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  """Entry point for launching an IPython kernel.


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

That was really cool!

It turns out that this idea can be taken *much* further.

In particular, instead of just incorporating a constant feature, we can incorporate arbitrary, nonlinear *functions* of the original input data.

For instance,

In [56]:
N     = 100 # number of datapoints
D     = 1   # dimension of datapoints
sigma = 0.5 # output noise
X     = t.randn(N, D)
qtrue = 2  # quadratic term
ltrue = -1 # linear term
btrue = 1  # bias
Y     = qtrue*X**2 + ltrue*X + btrue + sigma*t.randn(N, 1)

fig, ax = plt.subplots()
ax.set_xlabel("$x_\lambda$")
ax.set_ylabel("$y_\lambda$")
ax.scatter(X, Y);

/Users/laurence/programs/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  # Remove the CWD from sys.path while we load stuff.


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [57]:
def quad(X):
    return t.cat([X**2, X, t.ones(X.shape[0], 1)], 1)

Xe = quad(X)
Xe[:10, :]

tensor([[ 0.3204, -0.5660,  1.0000],
        [ 0.0230, -0.1518,  1.0000],
        [ 6.8478,  2.6168,  1.0000],
        [ 0.9868, -0.9934,  1.0000],
        [ 0.9389,  0.9690,  1.0000],
        [ 0.0296, -0.1721,  1.0000],
        [ 6.4896, -2.5475,  1.0000],
        [ 0.0751, -0.2740,  1.0000],
        [ 0.0469,  0.2166,  1.0000],
        [ 0.0121,  0.1102,  1.0000]])

In [58]:
Wh = fit_Wh(Xe, Y)
print(f"Wh    = {Wh}")
print(f"qtrue = {qtrue}")
print(f"ltrue = {ltrue}")
print(f"btrue = {btrue}")

Wh    = tensor([[ 2.0368, -1.0608,  0.9671]])
qtrue = 2
ltrue = -1
btrue = 1


In [59]:
fig, ax = plt.subplots()
ax.set_xlabel("$x_\lambda$")
ax.set_ylabel("$y_\lambda$")

ax.scatter(X, Y, label="data")

xs = t.linspace(-4, 4, 100)[:, None]
ax.plot(xs, quad(xs)@Wh.T, 'r', label="fitted line")
ax.legend();

/Users/laurence/programs/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  """Entry point for launching an IPython kernel.


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …